In [ ]:
import sys, os
nb_dir = os.getcwd()
# 2) Eine Ebene hoch → …/fairfluence/src
src_dir = os.path.abspath(os.path.join(nb_dir, os.pardir))
# 3) Noch eine Ebene hoch → …/fairfluence
project_root = os.path.abspath(os.path.join(src_dir, os.pardir))

# 4) Füge fairfluence/ nach ganz vorne in sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
import pandas as pd
from preprocessing import PreprocessorFactory, Preprocessor
import pytest
from datasets_dict import DATASETS
from src.ingestion.ingestorFactory import IngestorFactory

c:\Uni\Master\Semester_2\EDML\Project\fairfluence\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import sys, os
nb_dir = os.getcwd()
# 2) Eine Ebene hoch → …/fairfluence/src
src_dir = os.path.abspath(os.path.join(nb_dir, os.pardir))
# 3) Noch eine Ebene hoch → …/fairfluence
project_root = os.path.abspath(os.path.join(src_dir, os.pardir))

# 4) Füge fairfluence/ nach ganz vorne in sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
import pandas as pd
from preprocessing import PreprocessorFactory, Preprocessor
import pytest
from datasets_dict import DATASETS
from src.ingestion.ingestorFactory import IngestorFactory

def run_preprocessor_checks():
    methods = ["data quality", "fairness"]
    ohe_flags = [False, True]

    for index, data in enumerate(DATASETS.items()):
        data_link = data[0]
        info = data[1]
        if data_link in ["https://www.kaggle.com/datasets/aemreusta/brfss-2020-survey-data",
                        "https://www.kaggle.com/datasets/fleshmetal/records-a-comprehensive-music-metadata-dataset",
                        "https://www.kaggle.com/datasets/sudipde25/lifespan-calorie-burn-dynamics"]:
            continue
        target_column = info["target_column"]
        nr_file = info["no_dataset"]
        # 1) CSV laden
        ingestor_factory = IngestorFactory(data_link, nr_file, False)
        ingestor = ingestor_factory.create()
        df = ingestor.load_data()
        print(f"\n==> Testing {data_link!r}  target={target_column}")

        for method in methods:
            for ohe in ohe_flags:
                label = f"  [{method:12s} | ohe={ohe}]"
                try:
                    # 2) Factory + Preprocessor
                    factory = PreprocessorFactory(df, method, target_column)
                    pre     = factory.create()

                    # 3) Aufruf process_data
                    result = pre.process_data(ohe=ohe)

                    # 4) Struktur prüfen
                    if not isinstance(result, tuple):
                        raise AssertionError(f"{label} ➞ returned no tuple")
                    if len(result) != 6:
                        raise AssertionError(f"{label} ➞ expected 6 items, got {len(result)}")

                    df_tr, num_cols, cat_cols, text_feats, sens_cols, tgt_col = result

                    # 5a) DataFrame-Typ und Zeilenzahl
                    if not isinstance(df_tr, pd.DataFrame):
                        raise AssertionError(f"{label} ➞ first item not a DataFrame")
                    if df_tr.shape[0] != df.shape[0]:
                        raise AssertionError(f"{label} ➞ row count changed {df_tr.shape[0]} vs {df.shape[0]}")

                    # 5b) Target-Spalte vorhanden
                    if target_column not in df_tr.columns:
                        raise AssertionError(f"{label} ➞ target column '{target_column}' missing")
                    
                    # Nan Check
                    if df_tr.isnull().values.any():
                        nan_cols = df_tr.columns[df_tr.isnull().any()].tolist()
                        raise AssertionError(f"{label} ➞ found NaNs in columns {nan_cols}")

                    # 6) Die vier Listen müssen Listen sein
                    for name, lst in [
                        ("numeric_columns",    num_cols),
                        ("categorical_columns",cat_cols),
                        ("text_features",      text_feats),
                        ("sensitive_columns",  sens_cols)
                    ]:
                        if not isinstance(lst, list):
                            raise AssertionError(f"{label} ➞ '{name}' is not a list")

                    # 7) target_column-Return
                    if not isinstance(tgt_col, str) or tgt_col != target_column:
                        raise AssertionError(f"{label} ➞ returned target '{tgt_col}'") 
                    
                    # Every thing is numerical
                    if ohe:
                        for col in df_tr.columns:
                            if not pd.api.types.is_numeric_dtype(df_tr[col]):
                                raise AssertionError(
                                    f"{label} ➞ column '{col}' is not numeric (dtype={df_tr[col].dtype})"
                                )
                    
                    
                    vals = df_tr[target_column].unique()
                    if len(vals) != 2:
                        raise AssertionError(
                            f"{label} ➞ target column must be binary, "
                            f"but found values {vals}"
                        )

                    print(f"{label} ✓")
                except Exception as e:
                    print(f"{label} ✗ ERROR: {e}")

    print("\nAll checks done.")


run_preprocessor_checks()

c:\Uni\Master\Semester_2\EDML\Project\fairfluence\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Creating CSV from Arrow format: 100%|██████████| 33/33 [00:00<00:00, 448.85ba/s]
c:\Uni\Master\Semester_2\EDML\Project\fairfluence\src\preprocessing\preprocessing.py:109: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  categorical_columns = pd.unique(categorical_columns).tolist()



==> Testing 'https://huggingface.co/datasets/scikit-learn/adult-census-income'  target=income
  [data quality | ohe=False] ✓


c:\Uni\Master\Semester_2\EDML\Project\fairfluence\src\preprocessing\preprocessing.py:109: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  categorical_columns = pd.unique(categorical_columns).tolist()
c:\Uni\Master\Semester_2\EDML\Project\fairfluence\src\preprocessing\preprocessing.py:109: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  categorical_columns = pd.unique(categorical_columns).tolist()


  [data quality | ohe=True] ✓
  [fairness     | ohe=False] ✓


c:\Uni\Master\Semester_2\EDML\Project\fairfluence\src\preprocessing\preprocessing.py:109: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  categorical_columns = pd.unique(categorical_columns).tolist()


...

Creating CSV from Arrow format: 100%|██████████| 30/30 [00:00<00:00, 193.59ba/s]
c:\Uni\Master\Semester_2\EDML\Project\fairfluence\src\preprocessing\preprocessing.py:109: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  categorical_columns = pd.unique(categorical_columns).tolist()
c:\Uni\Master\Semester_2\EDML\Project\fairfluence\src\preprocessing\preprocessing.py:109: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  categorical_columns = pd.unique(categorical_columns).tolist()
c:\Uni\Master\Semester_2\EDML\Project\fairfluence\src\preprocessing\preprocessing.py:109: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  categorical_columns = pd.unique(categorical_columns).tolist()



==> Testing 'https://huggingface.co/datasets/scikit-learn/credit-card-clients'  target=default.payment.next.month
  [data quality | ohe=False] ✓
  [data quality | ohe=True] ✓


c:\Uni\Master\Semester_2\EDML\Project\fairfluence\src\preprocessing\preprocessing.py:109: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  categorical_columns = pd.unique(categorical_columns).tolist()


  [fairness     | ohe=False] ✓


KeyboardInterrupt: 

In [3]:
import sys, os
nb_dir = os.getcwd()
# 2) Eine Ebene hoch → …/fairfluence/src
src_dir = os.path.abspath(os.path.join(nb_dir, os.pardir))
# 3) Noch eine Ebene hoch → …/fairfluence
project_root = os.path.abspath(os.path.join(src_dir, os.pardir))

# 4) Füge fairfluence/ nach ganz vorne in sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
import pandas as pd
from preprocessing import PreprocessorFactory, Preprocessor
import pytest
from datasets_dict import DATASETS
from src.ingestion.ingestorFactory import IngestorFactory

data_link = "https://www.kaggle.com/datasets/ehtishamsadiq/uncleaned-laptop-price-dataset"
nr_file = 1
target_column = "Price"
ingestor_factory = IngestorFactory(data_link, nr_file, False)
ingestor = ingestor_factory.create()
data = ingestor.load_data()
dqp = Preprocessor(data, target_column="")
numeric_columns, categorical_columns = dqp.receive_categorized_columns()
preprocessor_factory = PreprocessorFactory(data, "fairness", target_column=target_column)
preprocessor = preprocessor_factory.create()
processed_data_dict = preprocessor.process_data(ohe=True)


processed_data_dict[0]

laptopData.csv
Dataset URL: https://www.kaggle.com/datasets/ehtishamsadiq/uncleaned-laptop-price-dataset
../../data/kaggle_temp/laptopData.csv was deleted.


,Company_<MISSING>,Company_Acer,Company_Apple,Company_Asus,Company_Chuwi,Company_Dell,Company_Fujitsu,Company_Google,Company_HP,Company_Huawei,...,OpSys_Mac OS X,OpSys_No OS,OpSys_Windows 10,OpSys_Windows 10 S,OpSys_Windows 7,OpSys_macOS,Unnamed: 0,Weight_svd_0,Weight_svd_1,Price
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,-1.720688,-5.395400e-07,-7.714979e-05,1
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,-1.720688,2.361558e-10,-3.895210e-07,0
2,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,-1.720688,-2.345284e-06,-2.149377e-05,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,-1.720688,2.496907e-11,1.410917e-08,1
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,-1.720688,-5.395400e-07,-7.714979e-05,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1.711762,3.982685e-09,3.332271e-06,0
1299,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1.711762,-3.471560e-10,1.000000e+00,1
1300,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1.711762,2.467374e-08,4.572228e-06,0
1301,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,1.711762,2.791961e-09,-2.539598e-07,0


In [15]:
target_column in numeric_columns

True

In [16]:
target_column in categorical_columns

False

In [4]:
processed_data_dict[4]

['AGE']

In [4]:
data

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,NaN,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,NaN,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


In [1]:
import sys, os
nb_dir = os.getcwd()
# 2) Eine Ebene hoch → …/fairfluence/src
src_dir = os.path.abspath(os.path.join(nb_dir, os.pardir))
# 3) Noch eine Ebene hoch → …/fairfluence
project_root = os.path.abspath(os.path.join(src_dir, os.pardir))

# 4) Füge fairfluence/ nach ganz vorne in sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
import pandas as pd
from preprocessing import PreprocessorFactory
import pytest
from datasets_dict import DATASETS
from src.ingestion.ingestorFactory import IngestorFactory

data_link = "https://www.kaggle.com/datasets/beridzeg45/usa-car-market"
nr_file = 1
target_column = "Price"
ingestor_factory = IngestorFactory(data_link, nr_file, False)
ingestor = ingestor_factory.create()
data = ingestor.load_data()
preprocessor_factory = PreprocessorFactory(data, "fairness", target_column=target_column)
preprocessor = preprocessor_factory.create()
processed_data_dict = preprocessor.process_data(ohe=True)
processed_data_dict[0]

c:\Uni\Master\Semester_2\EDML\Project\fairfluence\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cars_for_sale(cleaned).csv
Dataset URL: https://www.kaggle.com/datasets/beridzeg45/usa-car-market
../../data/kaggle_temp/cars_for_sale(cleaned).csv was deleted.
numeric_columns duplicates? False
text_columns    duplicates? False
categorical_columns duplicates? False
['Accidents or damage', '1-owner vehicle', 'Personal use only', 'Clean title', 'Open recall', 'Comfort_score', 'Interior_score', 'Performance_score', 'Value_score', 'Exterior_score', 'Reliability_score', 'Make', 'Condition', 'Interior color', 'Drivetrain', 'Fuel type']
Price <class 'pandas.core.series.Series'> 1
None


c:\Uni\Master\Semester_2\EDML\Project\fairfluence\src\preprocessing\preprocessing.py:109: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  categorical_columns = pd.unique(categorical_columns).tolist()


,Mileage_0.00–3.00,Mileage_3.00–7.00,Mileage_7.00–12.00,Mileage_12.00–4920.00,Mileage_4920.00–18944.00,Mileage_18944.00–33560.00,Mileage_33560.00–51202.00,Mileage_51202.00–71672.00,Mileage_71672.00–100562.50,Mileage_100562.50–324349.00,...,Fuel type_Hybrid,Fuel type_Plug-In Hybrid,Fuel type_–,Year,Seller Rating,Seller Rating Count,Min MPG,Max MPG,Engine Size (L),Price
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0.758068,0.517201,-0.057247,-0.924839,-0.225627,1.646751,1
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,-2.141767,-0.404393,-0.678082,0.198495,0.557029,-0.415303,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,-0.606560,-2.431901,-0.692450,0.038019,0.165701,0.695034,1
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,-0.606560,-1.510307,-0.917796,0.358971,0.035258,-0.732542,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,-0.265403,-0.035756,-0.941994,-3.331984,-3.486694,-0.732542,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9011,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0.758068,-0.773031,-0.969973,-0.443410,-0.225627,0.536414,1
9012,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0.416911,0.701520,1.816601,0.038019,0.165701,-0.732542,0
9013,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0.758068,0.148563,1.935323,-0.603886,-0.877840,1.884680,1
9014,0,0,0,0,0,0,0,0,0,1,...,0,0,0,-1.288874,0.332882,-0.262176,-0.122458,-0.616955,-0.415303,0


In [ ]:
processed_data_dict[0]["priceHistory/1/postingIsRental"]

KeyError: 'priceHistory/1/postingIsRental'

In [4]:
data["priceHistory/1/postingIsRental"].unique()

array([False, nan, True], dtype=object)

In [1]:
import sys, os
nb_dir = os.getcwd()
# 2) Eine Ebene hoch → …/fairfluence/src
src_dir = os.path.abspath(os.path.join(nb_dir, os.pardir))
# 3) Noch eine Ebene hoch → …/fairfluence
project_root = os.path.abspath(os.path.join(src_dir, os.pardir))

# 4) Füge fairfluence/ nach ganz vorne in sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
import pandas as pd
from preprocessing import PreprocessorFactory
import pytest
from datasets_dict import DATASETS
from src.ingestion.ingestorFactory import IngestorFactory

data_link = "https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset"
nr_file = 1
target_column = "stroke"
ingestor_factory = IngestorFactory(data_link, nr_file, False)
ingestor = ingestor_factory.create()
data = ingestor.load_data()
preprocessor_factory = PreprocessorFactory(data, "fairness", target_column=target_column)
preprocessor = preprocessor_factory.create()
processed_data_dict = preprocessor.process_data(ohe=True)
processed_data_dict[0]

c:\Uni\Master\Semester_2\EDML\Project\fairfluence\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


healthcare-dataset-stroke-data.csv
Dataset URL: https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset
../../data/kaggle_temp/healthcare-dataset-stroke-data.csv was deleted.


,gender_Female,gender_Male,gender_Other,age_0-9,age_10-19,age_20-29,age_30-39,age_40-49,age_50-59,age_60-69,...,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes,id,avg_glucose_level,bmi,stroke
0,0,1,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,-1.299046,2.729251,1.051577,0
1,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0.716749,2.139974,-0.095996,0
2,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,-0.255637,-0.002831,0.498042,0
3,1,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,1,1.118962,1.450556,0.754558,0
4,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,-1.648063,1.514869,-0.649531,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5105,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,-0.864584,-0.496194,-0.095996,1
5106,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0.395064,0.426220,1.510606,1
5107,1,0,0,0,0,0,0,1,0,0,...,1,0,0,0,1,0,-0.794175,-0.513107,0.241525,1
5108,0,1,0,0,0,0,0,0,0,1,...,1,0,0,1,0,0,0.048506,1.340623,-0.433518,1


In [2]:
pd.set_option("display.max_columns", None)